## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('../camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        cv2.imshow('img',img)
        cv2.waitKey(500)

cv2.destroyAllWindows()

## And so on and so forth...

In [5]:
# calibrate camera
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

[[1.15777930e+03 0.00000000e+00 6.67111054e+02]
 [0.00000000e+00 1.15282291e+03 3.86128938e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
[[-0.24688775 -0.02373133 -0.00109842  0.00035108 -0.00258571]]


In [2]:
import os
test_images = glob.glob('../test_images/*.jpg')

In [13]:
# undistort the test images
for fname in test_images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    if ret:
        cal_img = cv2.undistort(img, mtx, dist, None, mtx)
    else:
        print(fname, 'has a problem with calibration')
        cal_img = img.copy()
    cv2.imwrite(os.path.join('../output_images', 'cal_img_' + os.path.basename(fname)), cal_img)

In [18]:
cal_images = glob.glob('../output_images/cal_img_straight*.jpg') + glob.glob('../output_images/cal_img_test*.jpg')
for fname in cal_images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)    
    mag_threshold = (60, 255)
    R_threshold = (70, 255)
    S_threshold = (100, 255)  # more flexible
    #gaussian_k = 5
    #gray = cv2.GaussianBlur(gray, (gaussian_k, gaussian_k), 0)
    
    # magnitude of sobel operator
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    
    mag_sobel = np.sqrt(sobelx**2 + sobely**2)
    grad_dire = np.arctan2(abs(sobely), abs(sobelx))  # not good result
    abs_sobel = np.absolute(mag_sobel)
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    smag_bin_img = np.zeros_like(abs_sobel)
    smag_bin_img[(scaled_sobel >= mag_threshold[0]) & (scaled_sobel <= mag_threshold[1])] = 255
    
    # r-channel of BGR color space
    R = img[:, :, 2]
    sobelx_R = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    abs_sobel = np.absolute(sobelx_R)
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    R_bin_img = np.zeros_like(R)
    R_bin_img[(scaled_sobel >= R_threshold[0]) & (scaled_sobel <= R_threshold[1])] = 255
    
    # s-channel of HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    S = hls[:, :, 2]
    S_bin_img = np.zeros_like(S)
    S_bin_img[(S >= S_threshold[0]) & (S <= S_threshold[1])] = 255
    
    # stack channels and combine them to one binary image
    stacked_bin_img = np.dstack((smag_bin_img, S_bin_img, S_bin_img))
    combined_bin_img = np.zeros_like(abs_sobel)
    combined_bin_img[(smag_bin_img == 255) | (S_bin_img == 255)] = 255
    
    cv2.imwrite(os.path.join('../output_images', os.path.basename(fname).replace('cal_img', 'mag_S_thresholded')), combined_bin_img)
    #cv2.imshow('binary_img', stacked_bin_img)
    #cv2.waitKey(1500)

cv2.destroyAllWindows()